In [1]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import sqlite3

# establish sql connection
db_path = "data/input_data.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# PV Vergleich

In [6]:
df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])
df_station40 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_40', conn)
df_station43 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_43', conn)
df_station25 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_25', conn)
df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', conn, parse_dates=['Timestamp'])

df_mb_15 = df_mb_15.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
result_df = df_mb_15.merge(df_households, on='Timestamp', how='inner')

In [9]:
fig = make_subplots(specs=[[{'secondary_y': True}]])
for col in ['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla']:
    fig.add_trace(go.Scatter(x=result_df['Timestamp'], y=result_df[col], name=col, opacity=0.7), secondary_y=True)
fig.add_trace(go.Scatter(x=result_df['Timestamp'], y=result_df['pvpower_instant'], name='MeteoBlue', opacity=0.7), secondary_y=False)
fig.update_layout(template='plotly')
fig.update_yaxes(title_text='Household Batteries', secondary_y=True)
fig.update_yaxes(title_text='MeteoBlue', secondary_y=False)
fig.show(renderer='browser')

# PV Korrelation